# Comment entraîner proprement un modèle d’apprentissage

Jusqu’à présent, nous avons abordé les concepts essentiels de l’apprentissage supervisé très simplement, en les répartissant dans des étapes incontournables de tout projet de *machine learning*. De la constitution du jeu de données à l’entraînement du modèle en passant par la visualisation des interactions entre les variables explicatives et leur pré-traitement (recodage, mise à l’échelle, gestion des données manquantes…), il est en quelques manipulations possible d’obtenir des résultats satisfaisants avec les outils de *Scikit-Learn* dans la mesure où l’on est certain·es de disposer de données fiables et d’avoir fixé un objectif compréhensible.

La réalité est plus nuancée. Si notre volonté n’est pas de dresser un panorama exhaustif des techniques de paramétrage d’un modèle d’apprentissage et de leurs subtilités, pour cela nous renvoyons à des ouvrages plus complets comme celui de Aurélien Géron, [*Hands-on Machine Learning with Scikit-Learn, Keras and TensorFlow*](https://www.oreilly.com/library/view/hands-on-machine-learning/9781098125967/), notre intention est d’infuser un certain nombre de réflexes propres à éviter les principaux écueils inhérents aux méthodes statistiques.

Commençons par charger les librairies nécessaires :

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from math import sqrt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures

## Du problème de l’ajustement

Nous avons souvent évoqué les difficultés rencontrées par les modèles d’apprentissage pour s’ajuster aux données sans vraiment expliquer concrètement les deux cas de figure qui peuvent se présenter :
- Le **sous-ajustement** (*underfitting*), survenant lorsque le modèle ne parvient pas à percevoir la forme des données ;
- le **sur-ajustement** (*overfitting*), souvent caractérisé par des erreurs de généralisation bien plus importantes que celles d’entraînement.

L’objectif n’est donc pas d’obtenir le meilleur score sur le jeu d’entraînement ou, pire, sur le jeu de test, mais bien de trouver la zone idéale qui minimise l’écart entre les erreurs d’entraînement et de généralisation.

### Facteur de confusion

Avant d’entrer dans le vif du sujet, abordons un piège fondamental qui peut apparaître dès la définition du projet de *machine learning*. Imaginons que l’on souhaite modéliser un programme qui permette de prédire l’ampleur du bec d’un manchot et, comme variable explicative, nous retenons son poids.

Chargeons les données sur le recensement de trois espèces de manchots de l’Antarctique :

In [ ]:
df = pd.read_csv("./data/penguin-census.csv")
df = df.dropna()

Un modèle simple de régression linéaire nous apprend qu’il existe une corrélation négative entre les deux variables, si bien que le bec d’un manchot s’affine à mesure que sa masse corporelle augmente. Les chiffres ne mentent pas et, si l’on considère en prime l’intervalle de confiance à 95 %, la marge d’erreur est somme toute raisonnable :

In [ ]:
_ = sns.regplot(data=df, x="body_mass_g", y="bill_depth_mm")

La déduction nous semble malgré tout contre-intuitive. On aurait plutôt tendance à penser que les propriétés physiques de tout organisme biologique croissent proportionnellement à sa masse, non ?

Essayons de comprendre l’erreur de méthodologie que nous avons commise. Sur le graphique que nous venons d’afficher se distinguent deux groupes de points. Peut-être existe-t-il une différence entre les individus mâles et les individus femelles ?

In [ ]:
_ = sns.lmplot(data=df, x="body_mass_g", y="bill_depth_mm", col="sex", hue="sex")

Eh bien, non, cette hypothèse nous conforte dans notre erreur. Avant de supposer une différenciation de genre, rappelons-nous plutôt que la grande famille des manchots est subdivisée en plusieurs espèces avec des disparités physiques fortes et regardons le comportement de notre modèle linéaire à la lumière de ce nouveau facteur :

In [ ]:
_ = sns.lmplot(data=df, x="body_mass_g", y="bill_depth_mm", col="species", hue="species")

D’une corrélation négative entre la masse et l’épaisseur du bec d’un manchot nous sommes passés à une corrélation positive et avons évité de graves erreurs de généralisation au moment de la production de notre modèle.

Dans notre exemple, l’espèce d’appartenance d’un manchot est ce que l’on nomme un facteur de confusion, en ce sens qu’elle influe non seulement sur la variable cible, l’épaisseur du bec, mais aussi sur la variable explicative, la masse corporelle. En effet, si nous nous permettons une analyse grossière, un manchot papou ne peut pas peser moins de 4 kgs et son bec ne dépassera pas les 18 mm d’épaisseur pour les mieux lotis quand celui d’un manchot Adélie peut atteindre les 22 mm.

### Comparaison entre les erreurs d’entraînement et de généralisation

La compréhension des notions liées à l’ajustement d’un modèle passe par la comparaison des erreurs sur les jeux d’entraînement et de test. Prenons tout d’abord un ensemble de points à l’aspect sinusoïdal :

In [ ]:
# for deterministic purposes
np.random.seed(0)

# three hundred points
X = np.linspace(0, 10, 300).reshape(-1, 1)

# sinusoid function + noise
y = np.sin(X) + np.random.rand(300, 1)

coords = pd.DataFrame({
    'x': X[:, 0],
    'y': y[:, 0]
})

Si nous affichons un nuage de points, une forme se dessine clairement, nous laissant une vague idée sur la forme de la ligne qui devrait minimiser la fonction de coût :

In [ ]:
_ = sns.scatterplot(data=coords, x="x", y="y")

Constituons maintenant les jeux d’entraînement et de test selon une partition 80/20 et essayons d’entraîner dessus un modèle de régression linéaire que nous savons d’ores et déjà voué à l’échec :

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(coords["x"].values.reshape(-1,1), coords["y"], test_size=0.2, random_state=42)

model = LinearRegression()
_ = model.fit(X_train, y_train)

Les prédictions sont désastreuses :

In [ ]:
# predictions
y_pred = model.predict(X_test)

# plot test points
ax = sns.scatterplot(x=X_test[:, 0], y=y_test)
ax.plot(X_test, y_pred, color="orange")
_ = ax.set_title(f"R2 scores: train = {model.score(X_train, y_train):.3f} ; test = {model.score(X_test, y_test):.3f}")

D’un côté, en étant inférieur sur les données de test, le $R^2$ score est conforme à nos attentes ; d’un autre côté, il est révélateur d’un modèle sévèrement sous-ajusté.

Confirmons ce résultat en calculant maintenant l’erreur absolue moyenne (*mean absolute error*) et rappelons que si un score doit être le plus élevé possible, une erreur doit quant à elle être la plus faible possible :

In [ ]:
mae_train = mean_absolute_error(y_train, model.predict(X_train))
mae_test = mean_absolute_error(y_test, y_pred)

print(
    f"MAE (jeu d’entrainement) : {mae_train:.3f}",
    f"MAE (jeu de test) : {mae_test:.3f}",
    sep="\n"
)

L’erreur sur le jeu de test est encore plus importante que sur le jeu d’entraînement : aucun doute possible, le modèle est sous-ajusté.

Reprenons nos efforts en entraînant désormais un modèle de régression polynomial à haut degré :

In [ ]:
model = make_pipeline(
    PolynomialFeatures(degree=25),
    LinearRegression(),
)
model.fit(X_train, y_train)

y_train_predict = model.predict(X_train)
y_test_predict = model.predict(X_test)

mae_train = mean_absolute_error(y_train, y_train_predict)
mae_test = mean_absolute_error(y_test, y_test_predict)

La MAE sur le jeu d’entraînement reste élevée tandis qu’elle a baissé sur le jeu de test au point de lui être inférieure :

In [ ]:
figure, ax = plt.subplots()

sns.scatterplot(x=X_test[:, 0], y=y_test, ax=ax)
sns.lineplot(x=X_test[:, 0], y=y_test_predict, color="orange", ax=ax)

ax.set_title(f"MAE: train = {mae_train:.3f} ; test = {mae_test:.3f}")

plt.show()

On s’attendrait en fait à l’inverse d’un modèle justement entraîné, l’idée qu’il soit moins performant sur des données nouvelles étant tout à fait naturelle. En tâtonnant un peu, on finirait par trouver qu’un polynôme de degré 7 ou 8 constitue le meilleur choix.

## Entraîner, tester, valider

## Régulariser un modèle

## Sauvegarder